## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [201]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0170.shtml"

Устанавливаем библиотеки

In [202]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

Создаём объект морфологического анализатора `RNNMorph`

In [203]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [204]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текст с html тегами

In [205]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Упырь</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\nУпырь</h2>'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [206]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [207]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Упырь\n\n\n\nТолстой Алексей Константинович\r\nУпырь\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n'

In [208]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /Users/muzafarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 1464 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [209]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



23/23 [==============================] - 5s 185ms/step


sentences: 100%|██████████| 1464/1464 [00:00<00:00, 619560.19it/s]


[['слышать', 'ли', '?']]

In [210]:
import re

def clean_string(string):
    # Удаляем английские слова
    string = re.sub(r'\b[a-zA-Z]+\b', '', string)
    # Удаляем все символы кроме русских слов, цифр и некоторых специальных символов
    return re.sub(r'[0-9a-z.,!?:;"\'()\[\]#*@$&--^+-/{}|<>]', '', string)

!@#$^&%*()+=-[]\/{}|:<>?,.
# Function to process nested lists and remove empty elements
def process_list(lst):
    i = 0
    while i < len(lst):
        if isinstance(lst[i], list):
            process_list(lst[i])
            if not lst[i]:
                lst.pop(i)
                continue
        elif isinstance(lst[i], str):
            cleaned_item = clean_string(lst[i])
            if cleaned_item:
                lst[i] = cleaned_item
            else:
                lst.pop(i)
                continue
        i += 1


# Clean the original list
process_list(predictions)

print(predictions)

zsh:1: command not found: @#
zsh:1: no matches found: %*
zsh:1: no matches found: ?,.


[['классика', 'толстый', 'алексей', 'константинович'], ['упырь', 'толстый', 'алексей', 'константинович', 'упырь', 'классик', 'регистрация', 'найти', 'рейтинг', 'обсуждение', 'новинка', 'обзор', 'помощь', 'комментарий', 'последний', 'от'], ['толстый', 'алексей', 'константинович', 'год', 'обновлённый'], ['статистика'], ['повесть', 'проза', 'фантастика', 'проза', 'скачать', 'оценка', 'ваш', 'оценка', 'шедевр', 'замечательный', 'очень', 'хорошо', 'хорошо', 'нормально', 'не', 'читать', 'терпимо', 'посредственно', 'плохо', 'очень', 'плохо', 'не', 'читать', 'алексей', 'константинович', 'толстой'], ['упырь', 'оригинал', 'здесь', 'библиотека', 'альдебаран', 'бал', 'быть', 'очень', 'многолюдный'], ['после', 'шумный', 'вальс', 'руневский', 'отвести', 'свой', 'дама', 'на', 'он', 'место', 'и', 'стать', 'прохаживаться', 'по', 'комната', 'посматривать', 'на', 'различный', 'группа', 'гость'], ['он', 'броситься', 'в', 'глаз', 'человек', 'повидимому', 'ещё', 'молодой', 'но', 'бледный', 'и', 'почти', 'со

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [211]:
len(predictions)

1460

In [212]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


20979

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [213]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muzafarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [214]:
freq_dist = FreqDist(non_uniq_tokens)
freq_dist

FreqDist({'он': 1241, 'и': 785, 'я': 620, 'в': 561, 'не': 488, 'что': 439, 'быть': 295, 'на': 290, 'с': 270, 'вы': 233, ...})

In [215]:
# Отсортировать токены по убыванию частоты
sorted_tokens = sorted(freq_dist.items(), key=lambda x: x[1], reverse=True)

# Взять 100 самых частотных токенов
top_100_tokens = [token for token, freq in sorted_tokens[:100]]

# Посчитать, сколько из этих 100 токенов не являются стоп-словами
non_stop_words_count = sum(1 for token in top_100_tokens if token not in STOPWORDS)

# Рассчитать долю токенов, не относящихся к стоп-словам, среди 100 самых частотных токенов
proportion_non_stop_words = non_stop_words_count / len(top_100_tokens)

proportion_non_stop_words

0.48

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)


In [216]:
# Подсчет токенов, встречающихся строго больше 50 раз
tokens_over_50_freq = sum(1 for token, freq in freq_dist.items() if freq > 50)

tokens_over_50_freq

53

Введите количество токенов, состоящих только из букв

In [221]:
# Инициализация переменной для подсчета токенов, состоящих только из букв
letter_tokens_count = 0

# Перебираем все предложения
for sentence in tokenized_sentences:
    # Перебираем все токены в предложении
    for token in sentence:
        # Если токен состоит только из букв, увеличиваем счетчик
        if token.isalpha():
            letter_tokens_count += 1

# Выводим результат
print("Количество токенов, состоящих только из букв:", letter_tokens_count)


Количество токенов, состоящих только из букв: 20870


Какую долю среди 40 самых частотных токенов в произведении занимают слова, не входящие в стоп-лист?


In [218]:
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Список всех токенов
non_uniq_tokens

# Подсчет частоты встречаемости каждого токена
freq_dist = FreqDist(non_uniq_tokens)

# Отсортировать токены по убыванию частоты
sorted_tokens = sorted(freq_dist.items(), key=lambda x: x[1], reverse=True)

# Взять 40 самых частотных токенов
top_40_tokens = [token for token, freq in sorted_tokens[:40]]

# Посчитать, сколько из этих 40 токенов не входят в стоп-лист
non_stop_words_count = sum(1 for token in top_40_tokens if token not in stop_words)

# Рассчитать долю слов, не входящих в стоп-лист, среди 40 самых частотных токенов
proportion_non_stop_words = non_stop_words_count / 40

print("Доля слов, не входящих в стоп-лист, среди 40 самых частотных токенов:", proportion_non_stop_words)


Доля слов, не входящих в стоп-лист, среди 40 самых частотных токенов: 0.25


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muzafarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Сколько токенов встречается в тексте строго больше 10 раз?


In [219]:
# Подсчет токенов, встречающихся строго больше 10 раз
tokens_over_50_freq = sum(1 for token, freq in freq_dist.items() if freq > 10)

tokens_over_50_freq

279